In [1]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r"E:\gitlab\geoloc2\Detecciondeterrenos")
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r"C:\Users\ASUS\Inteligencia_Artificial\yolov7")
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image

In [2]:
import fiona
import rasterio
from rasterio.mask import mask



In [3]:
Modelo=modelo(weights="Modelos/best.pt")#,device="cpu") ## device="0" para gpu

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



## Parametros del raster

In [4]:
raster=r"C:\Users\ASUS\Desktop\neza1.tif"

In [5]:
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3]
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs,H,W,minx,maxx,miny,maxy,"W_metros",maxx-minx,"H_metros",maxy-miny

(CRS.from_epsg(3857),
 155650,
 112773,
 -11028184.223,
 -11016906.9436,
 2198108.5665,
 2213673.5468,
 'W_metros',
 11277.279399998486,
 'H_metros',
 15564.980299999937)

In [6]:
dim=int(np.ceil(map_d(minx+120,minx,maxx,0,W)))
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
alto,ancho,alto*ancho
nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
angulosget=[]
conf_casas=[]
conf_terreno=[]
alto,ancho,alto*ancho,dim

(129, 93, 11997, 1201)

## Generar prediccion sobre raster(municipio) con salida en shp

In [7]:
## shape igecem de manzanas por municipio (solo pora tener la delimitacion geografica del municipio)
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Nezahualcoyotl.shp")
shape=shape.to_crs("3857")
c=[]
for manzana in range(len(shape)):
    proyecciones1=mapping(shape["geometry"][manzana]).get("coordinates")
    if len(proyecciones1)>1:
        for manzan in proyecciones1:
            print(manzana)
            try:
                proyecciones=manzan
            except:
                proyecciones=manzan[0]
            point1=np.min((proyecciones,proyecciones),axis=1)[0]
            min_y,min_x=point1[0],point1[1]
            point2=np.max((proyecciones,proyecciones),axis=1)[0]
            max_y,max_x=point2[0],point2[1]
            c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
    else:
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones    
    
    # point1=np.min((proyecciones,proyecciones),axis=1)[0]
    # min_y,min_x=point1[0],point1[1]
    # point2=np.max((proyecciones,proyecciones),axis=1)[0]
    # max_y,max_x=point2[0],point2[1]
    # c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
shape1=pd.DataFrame()
shape1["points"]=c
shape1=shape1["points"].str.split(",",expand=True)
shape1=shape1.astype({0:"float64",1:"float64",2:"float64",3:"float64"})
shape=shape1

In [8]:
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Nezahualcoyotl.shp")
shape=shape.to_crs("3857")
shape["centroid"]=shape.centroid
shape
geometry=[]
for i,polygon in enumerate(shape["geometry"]):
    point=mapping(shape["centroid"][i]).get("coordinates")
    # print(point)
    x=point[0]
    y=point[1]
    go=[]
    coodinates=mapping(polygon).get("coordinates")[0]
    for a in coodinates:
        x1=a[0]
        y1=a[1]
        x2=x+(x1-x)*1.2
        y2=y+(y1-y)*1.2
        go.append((x2,y2))
    geometry.append(Polygon(go))
shape_aumentado=gpd.GeoDataFrame(geometry=geometry)#.to_file("C:/Users/ASUS/Desktop/shpa.sph")

In [9]:
shape_aumentado["geometry"]=shape_aumentado["geometry"].envelope
sa=[]
for i in range(len(shape_aumentado)):
    sa.append(len(mapping(shape_aumentado["geometry"][i]).get("coordinates")[0]))
shape_aumentado["sa"]=sa
shape_aumentado.to_file(r"C:\Users\ASUS\salida\h.sph\h.shp") #[shape["sa"]==5]

In [10]:
with rasterio.open(raster) as src:
    for s in tqdm.tqdm(range(len(shape_aumentado))):
        # try:
        if gdal.Open( r"D:/neza/CAT-"+str(shape["cve_cat"][s])+".tif")!=None:
            continue
        shapes=[shape_aumentado["geometry"][s]]
        try:
            manzana,out_transform = rasterio.mask.mask(src, shapes, crop=True)
        except:
            continue
        out_meta = src.meta

        out_meta.update({"driver": "GTiff",
        "height": manzana.shape[1],
        "width": manzana.shape[2],
        "transform": out_transform})

        output_file = r"D:/neza/CAT-"+str(shape["cve_cat"][s])+".tif"

        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(manzana)
        # except:
        #     pass

In [ ]:
out_meta

In [ ]:
shape["cve_cat"]

In [ ]:
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3]
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs,H,W,minx,maxx,miny,maxy,"W_metros",maxx-minx,"H_metros",maxy-miny

In [ ]:
imshow=False
with rasterio.open(raster) as src:
    xsize, ysize = dim,dim
    xmin, xmax = 0, src.width - xsize
    ymin, ymax = 0, src.height - ysize
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(ancho*2):#ancho
            for i in range(alto*2):#alto
                generar=0
                label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
                nameimg=label.lower()+str(i)+"_"+str(j)
                nombres.append(nameimg)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j/2+k),
                                    maxy-(maxy-miny)/alto*(i/2+l)))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
                    if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                        generar=1             
                if generar==1:
                    shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                    vector=[]
                    xoff, yoff = xsize*j,ysize*i
                    window = Window(xoff, yoff, xsize, ysize)
                    array=src.read(window=window)
                    # array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    four_images=[array[2],array[1],array[0]]
                    imagen_n = np.stack(four_images, axis=-1)
                    # if dim>256:
                    #     imagen_n=cv2.resize(imagen_n,(256,256))
                    #     dim=256
                    angulo_f=0
                    anterior=0
                    actual=1
                    vector1=[]
                    angulo=0
                    le=0
                    # try:
                    angulo_1=correct_orientation(imagen_n,dim)[0]
                    # except:
                    #     pbar.update(1)
                    #     continue
                    for var_angle in range(-2,2,1):
                        angulo=angulo_1+var_angle
                        anterior=len(vector)
                        image_ro=imagen_n.copy()
                        M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
                        image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
                        with torch.no_grad():
                            vector1=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.25,imagen_s=image_ro)
                        actual=len(vector)
                        if len(vector1)>le:
                            le=len(vector1)
                            vector=vector1
                            imagen_n1=image_ro
                            angulo_f=angulo        
                    # vector2=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.03,imagen_s=imagen_n)
                    # if len(vector)>len(vector2):
                    #     imagen_n1=image_ro
                    #     angulo_f=angulo
                    # else:
                    #     le=len(vector2)
                    #     vector=vector2
                    #     imagen_n1=imagen_n
                    #     angulo_f=0
                    angulo=angulo_f
                    # if angulo!=0:
                    #     imshow=True
                    angulosget.append(angulo)
                    proyecciones=shapes[0].get("coordinates")[0][:-1]
                    df_cache=vector2xy(vector,dim=dim,nameimg=nameimg)
                    for cs_1 in (range(len(df_cache))):
                        if df_cache["Tipo"][cs_1]=="casa":
                            casas.append(rotacion_detect(df_cache.loc[cs_1,"start_point_100"], df_cache.loc[cs_1,"end_point_100"],-angulo,proyecciones=proyecciones))
                            conf_casas.append(df_cache.loc[cs_1,"conf"])
                        else:
                            terreno.append(rotacion_detect(df_cache.loc[cs_1,"start_point_100"], df_cache.loc[cs_1,"end_point_100"],-angulo,proyecciones=proyecciones))
                            conf_terreno.append(df_cache.loc[cs_1,"conf"])
                    if len(casas)%1000==0:
                        gdf_casas=gpd.GeoDataFrame({"conf":conf_casas},geometry=casas,crs=src.crs)
                        gdf_casas.set_crs=src.crs
                        gdf_casas["area"]=gdf_casas.area
                        gdf_casas=gdf_casas.astype({"conf":"float64"})
                        # gdf_casas=gdf_casas[(gdf_casas["conf"]<0.5)&(gdf_casas["conf"]>.09)]
                        gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/neza2_new_casas.shp")
                        print("generar casas")
                    if len(terreno)%1000==0:
                        gdf_terreno=gpd.GeoDataFrame({"conf":conf_terreno},geometry=terreno,crs=src.crs)
                        gdf_terreno.set_crs=src.crs
                        gdf_terreno.to_file(r"C:/Users/ASUS/salida/shape/neza2_new_terreno.shp")
                        print("generar terreno")
                    if imshow:
                        print(angulo)
                        imshow_detect(df_cache,imagen_n1)
                        # imshow=False

                pbar.update(1)
                


casas

In [ ]:
gdf_casas=gpd.GeoDataFrame({"conf":conf_casas},geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas.astype({"conf":"float64"})
# gdf_casas=gdf_casas[(gdf_casas["conf"]<0.5)&(gdf_casas["conf"]>.09)]
gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/neza2_new_casas.shp")
gdf_terreno=gpd.GeoDataFrame({"conf":conf_terreno},geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:/Users/ASUS/salida/shape/neza2_new_terreno.shp")

In [ ]:
### generar malla de corte en raster
geometry=[]
with tqdm.tqdm(total=alto*ancho) as pbar:
    for j in range(ancho):#ancho
        for i in (range(alto)):#alto
            k = cv2.waitKey(1)
            if k == ord('s'):
                print("entre aqui")
            generar=0
            label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
            nameimg=label.lower()+str(i)+"_"+str(j)
            nombres.append(nameimg)
            cuadro=[]
            for k in range(2):
                for l in range(2):
                    cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                maxy-(maxy-miny)/alto*(i+l)))
                    
            cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
            geometry.append(Polygon(cuadro))
            pbar.update(1)
gdf=gpd.GeoDataFrame(geometry=geometry,crs=3857).to_file("C:/Users/ASUS/salida/shape/cutline.shp")

In [ ]:
coodinates=mapping(shape["geometry"][1]).get("coordinates")[0]
p1=coodinates[0]
p2=coodinates[1]
p3=coodinates[2]
d1=np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
d2=np.sqrt((p2[0]-p3[0])**2+(p2[1]-p3[1])**2)
x_n,y_n=p2[0]+10*np.abs(p3[0]-p2[0])/d2,p2[1]+10*np.abs(p3[1]-p2[1])/d2
x_n1,y_n1=p2[0]+10*np.abs(p1[0]-p2[0])/d1,p2[1]+10*np.abs(p1[1]-p2[1])/d1
gpd.GeoDataFrame(geometry=[gen_cuadro(p2[0],p2[1],x_n,y_n,x_n+10,y_n+10,x_n1,y_n1)],crs=3857).to_file("C:/Users/ASUS/Desktop/shpa.shp")


In [ ]:
def gen_cuadro(x1,y1,x2,y2,x3,y3,x4,y4):
    return(Polygon(((x1,y1),(x2,y2),(x3,y3),(x4,y4),(x1,y1))))

In [ ]:
from rasterio.transform import from_origin

src=rasterio.open(r"D:/haar/classes/carros.tif",transform=from_origin(-180.0, 90.0, 0.5, 0.5))
src.read().shape


In [ ]:
rasterio.transform

## Generar imagenes mosaico

In [ ]:
# Path donde se guardan images
path_mosaico_salida=r"D:\instream\Mi unidad\geoshaps\neza\images/"
generar_imagenes=int(input("Generar imagenes: 1 si, 0 no "))
if generar_imagenes==1:
    generar_imagenes_sinrotar=int(input("Generar imagenes sin rotar: 1 si, 0 no "))
    generar_imagenes_rotadas=int(input("Generar imagenes rotadas: 1 si, 0 no "))
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(0,ancho):#ancho
            for i in (range(alto)):#alto
                generar=0
                label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
                nameimg=label.lower()+str(i)+"_"+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                       maxy-(maxy-miny)/alto*(i+l),
                                       0.0))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
                    if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                        generar=1             
                if generar==1:
                    shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    four_images=[array[2],array[1],array[0]]
                    imagen_n = np.stack(four_images, axis=-1)
                    if generar_imagenes_sinrotar==1:
                        cv2.imwrite(path_mosaico_salida+nameimg+".png",imagen_n)
                    if generar_imagenes_rotadas==1:
                        angulo_1,imagen_ro=correct_orientation(imagen_n,dim)
                        cv2.imwrite(path_mosaico_salida+nameimg+"_"+str(angulo_1)+".png",imagen_ro)
                pbar.update(1)

## Generar prediccion sobre imagen con salida txt

In [ ]:
# path_mosaico_salida="D:/tlalne/"
vector=Modelo.detect(opt_source=path_mosaico_salida,opt_conf_thres=0.08)
Generar_txt(vector,path_mosaico_salida+"labels")
with open(path_mosaico_salida+"labels/classes.txt","w") as f:
    f.writelines("\n".join(["casa","terreno_baldio"]))

In [ ]:
## shape igecem de manzanas por municipio (solo pora tener la delimitacion geografica del municipio)
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Naucalpan.shp")
shape=shape.to_crs("3857")
c=[]
for manzana in range(len(shape)):
    proyecciones1=mapping(shape["geometry"][manzana]).get("coordinates")
    if len(proyecciones1)>1:
        for manzan in proyecciones1:
            print(manzana)
            try:
                proyecciones=manzan
            except:
                proyecciones=manzan[0]
            point1=np.min((proyecciones,proyecciones),axis=1)[0]
            min_y,min_x=point1[0],point1[1]
            point2=np.max((proyecciones,proyecciones),axis=1)[0]
            max_y,max_x=point2[0],point2[1]
            c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
    else:
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones    
    
    # point1=np.min((proyecciones,proyecciones),axis=1)[0]
    # min_y,min_x=point1[0],point1[1]
    # point2=np.max((proyecciones,proyecciones),axis=1)[0]
    # max_y,max_x=point2[0],point2[1]
    # c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
shape1=pd.DataFrame()
shape1["points"]=c
shape1=shape1["points"].str.split(",",expand=True)
shape1=shape1.astype({0:"float64",1:"float64",2:"float64",3:"float64"})
shape=shape1
shape

In [ ]:
from shapely.geometry import Point
geo=[]
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Naucalpan.shp")
shape=shape.to_crs("3857")
c=[]
for manzana in range(len(shape)):
    proyecciones1=mapping(shape["geometry"][manzana]).get("coordinates")
    for i in proyecciones1[0]:
        geo.append(Point(i))
sa=gpd.GeoDataFrame(geometry=geo)
sa

In [ ]:
sa.to_file(r"C:\Users\ASUS\Desktop\nau\pointnau.shp")

In [ ]:
gpd.sjoin(shape,shape1 , how='left', op='intersects').to_file(r"C:\Users\ASUS\Desktop\nau\sol.shp")

In [ ]:
shape1=gpd.read_file(r"C:\Users\ASUS\Desktop\nau\new_terreno1.shp")z
shape1

In [ ]:
indices=[]
for i,pol in enumerate(shape1["geometry"]):
    generar=0
    cuadro=mapping(pol).get("coordinates")[0]
    for punto in cuadro:
            x=float(punto[0])
            y=float(punto[1])
            if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                generar=1             
    if generar==1:
         indices.append(i)
indices

In [ ]:
shape1.loc[indices].to_file(r"C:\Users\ASUS\Desktop\nau/solo_naucalpan.shp")

In [ ]:
gpd.sjoin(shape,shape1 , how='inner', op='intersects').to_file(r"C:\Users\ASUS\Desktop\nau/sol.shp")

In [ ]:
complete_polys = [polygon for polygon in  if (polygon.centroid.x**2 + polygon.centroid.y**2 < 4**2)]

complete_polys

In [ ]:
import random
import rasterio
from rasterio.windows import Window

with rasterio.open(r"D:\haar\classes\carros.tif") as src:

    # The size in pixels of your desired window
    xsize, ysize = 1000,1000

    # Generate a random window origin (upper left) that ensures the window 
    # doesn't go outside the image. i.e. origin can only be between 
    # 0 and image width or height less the window width or height
    xmin, xmax = 0, src.width - xsize
    ymin, ymax = 0, src.height - ysize
    xoff, yoff = random.randint(xmin, xmax), random.randint(ymin, ymax)

    # Create a Window and calculate the transform from the source dataset    
    window = Window(0, 0, xsize, ysize)
    transform = src.window_transform(window)

    # Create a new cropped raster to write to
    profile = src.profile
    profile.update({
        'height': xsize,
        'width': ysize,
        'transform': transform})

    with rasterio.open('D:/haar/classes/carros_output.tif', 'w', **profile) as dst:
        # Read the data from the window and write it to the output raster
        dst.write(src.read(window=window))